### Workflow: Practice SQL with Python

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect(':memory:')  # creates a temporary DB in RAM
cursor = conn.cursor()

In [3]:
# create some tables

cursor.execute('''
CREATE TABLE employees (
    id INTEGER PRIMARY KEY,
    name TEXT,
    department TEXT,
    salary INTEGER,
    hire_date TEXT
)
''')


In [4]:
# insert some data

data = [
    (1, 'Alice', 'Engineering', 90000, '2020-05-01'),
    (2, 'Bob', 'Marketing', 65000, '2019-03-15'),
    (3, 'Charlie', 'HR', 70000, '2018-07-23'),
    (4, 'Diana', 'Engineering', 95000, '2021-01-11'),
    (5, 'Eve', 'Marketing', 62000, '2020-12-01')
]

cursor.executemany('INSERT INTO employees VALUES (?, ?, ?, ?, ?)', data)
conn.commit()


In [5]:
# Example query: get all Engineering employees with salary > 80000
query = '''
SELECT name, salary FROM employees
WHERE department = 'Engineering' AND salary > 80000
ORDER BY salary DESC
'''

df = pd.read_sql_query(query, conn)
print(df)

    name  salary
0  Diana   95000
1  Alice   90000


In [9]:
query = '''
SELECT name, salary FROM employees
WHERE department = 'Marketing'
ORDER BY salary DESC
'''

df = pd.read_sql_query(query, conn)
print(df)

  name  salary
0  Bob   65000
1  Eve   62000


Basic 2

In [11]:
import sqlite3
import pandas as pd

# Create in-memory SQLite DB
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create "employees" table
cursor.execute('''
CREATE TABLE employees (
    emp_id INTEGER PRIMARY KEY,
    emp_name TEXT,
    department_id INTEGER,
    salary INTEGER
)
''')

# Create "departments" table
cursor.execute('''
CREATE TABLE departments (
    department_id INTEGER PRIMARY KEY,
    department_name TEXT
)
''')

# Insert data into "departments"
departments_data = [
    (1, 'Engineering'),
    (2, 'Marketing'),
    (3, 'HR'),
    (4, 'Sales')
]
cursor.executemany('INSERT INTO departments VALUES (?, ?)', departments_data)

# Insert data into "employees"
employees_data = [
    (1, 'Alice', 1, 90000),
    (2, 'Bob', 2, 65000),
    (3, 'Charlie', 3, 70000),
    (4, 'Diana', 1, 95000),
    (5, 'Eve', 2, 62000),
    (6, 'Frank', None, 50000)  # No department assigned
]
cursor.executemany('INSERT INTO employees VALUES (?, ?, ?, ?)', employees_data)

conn.commit()


Fiest print all the db

In [14]:
query = '''
SELECT * FROM employees
'''

df = pd.read_sql_query(query, conn)
print(df)

   emp_id emp_name  department_id  salary
0       1    Alice            1.0   90000
1       2      Bob            2.0   65000
2       3  Charlie            3.0   70000
3       4    Diana            1.0   95000
4       5      Eve            2.0   62000
5       6    Frank            NaN   50000


In [15]:
query = '''
SELECT * FROM departments
'''

df = pd.read_sql_query(query, conn)
print(df)

   department_id department_name
0              1     Engineering
1              2       Marketing
2              3              HR
3              4           Sales


In [16]:
# how to limit the query

query = '''
SELECT * FROM employees
LIMIT 3
'''

df = pd.read_sql_query(query, conn)
print(df)

   emp_id emp_name  department_id  salary
0       1    Alice              1   90000
1       2      Bob              2   65000
2       3  Charlie              3   70000


Basic SELECT + WHERE + ORDER BY

In [19]:
query = '''
SELECT emp_name,salary FROM employees
WHERE department_id = 1 AND salary > 90000
ORDER BY salary DESC 
'''
print(pd.read_sql_query(query, conn))


  emp_name  salary
0    Diana   95000


GROUP BY

In [21]:
query = '''
SELECT department_id, AVG(salary) as avg_salary
FROM employees
GROUP BY department_id
ORDER BY avg_salary DESC

'''
print(pd.read_sql_query(query, conn))

   department_id  avg_salary
0            1.0     92500.0
1            3.0     70000.0
2            2.0     63500.0
3            NaN     50000.0


LEFT JOIN

In [23]:
query = '''
SELECt e.emp_name, d.department_name
FROM employees e
LEFT JOIN departments d
ON e.department_id = d.department_id
'''
print(pd.read_sql_query(query, conn))


  emp_name department_name
0    Alice     Engineering
1      Bob       Marketing
2  Charlie              HR
3    Diana     Engineering
4      Eve       Marketing
5    Frank            None


In [25]:
query = '''
SELECT e.emp_name, e.salary, d.department_name
FROM employees e
INNER JOIN departments d
ON e.department_id = d.department_id
'''
print(pd.read_sql_query(query, conn))


  emp_name  salary department_name
0    Alice   90000     Engineering
1      Bob   65000       Marketing
2  Charlie   70000              HR
3    Diana   95000     Engineering
4      Eve   62000       Marketing


Subquery Example

In [27]:
# Goal: Get employees who earn more than the average salary.

query = '''
SELECT emp_name, salary
FROM employees
WHERE salary > (
    SELECT AVG(salary)
    FROm employees
)
'''
print(pd.read_sql_query(query, conn))



  emp_name  salary
0    Alice   90000
1    Diana   95000


In [28]:
query = '''

    SELECT AVG(salary)
    FROm employees

'''
print(pd.read_sql_query(query, conn))

   AVG(salary)
0      72000.0
